In [11]:
# Spider https://huggingface.co/datasets/spider
import pandas as pd

df_parquet = pd.read_parquet('/Users/ivanbondyrev/Downloads/train-00000-of-00001.parquet')
df_parquet_test = pd.read_parquet('/Users/ivanbondyrev/Downloads/validation-00000-of-00001.parquet')

with open('cleaned_data/Spider/spider_train.txt', "w+") as file:
    for line in df_parquet['query']:
        file.write(line + "\n")

with open('cleaned_data/Spider/spider_test.txt', "w+") as file:
    for line in df_parquet_test['query']:
        file.write(line + "\n")

In [15]:
# WikiSQL https://www.kaggle.com/datasets/shahrukhkhan/wikisql?resource=download&select=validation.csv
# NB: very bad dataset, maybe do not use it

import pandas as pd

def write_to_file(name: str, df):
    with open(f"cleaned_data/WikiSQL/{name}.txt", "w+") as file:
        for line in df['sql']:
            file.write(line + '\n')

df_test = pd.read_csv('/Users/ivanbondyrev/Downloads/test.csv')
df_validation = pd.read_csv('/Users/ivanbondyrev/Downloads/validation.csv')
df_train = pd.read_csv('/Users/ivanbondyrev/Downloads/train.csv')

write_to_file('train', df_train)
write_to_file('test', df_test)
write_to_file('validation', df_validation)


In [1]:
# Clickhouse generated 

import re
with open('/Users/ivanbondyrev/Downloads/query_log.tsv', 'r', encoding='latin-1') as file, open("dataset_tests_pr_60095.txt", "w") as f:
    for line in file:
        if line.split("\t")[1] != "QueryFinish":
            continue
        string = line.split("\t")[16].encode().decode('unicode_escape')
        string = re.sub(r'\s+', ' ', string)
        if string == "SELECT 1":
            continue
        if string.startswith("--"):
            continue
        f.write(string)
        f.write("\n")



In [2]:
# split clickhouse generated to train/test/split
import random

filename = "cleaned_data/Clickhouse/clickhouse_tests_pr_60095.txt"
train_filename = "cleaned_data/Clickhouse/clickhouse_train_pr_60095.txt"
test_filename = "cleaned_data/Clickhouse/clickhouse_test_pr_60095.txt"
val_filename = "cleaned_data/Clickhouse/clickhouse_validation_pr_60095.txt"
with open(filename, "r") as f:
    lines = list(set(f.readlines()))
    random.shuffle(lines)

    train_idx = int(len(lines) * 0.75)
    test_idx = int(len(lines) * 0.95)
    
    with open(train_filename, "w+") as train_f:
        train_f.writelines(lines[:train_idx])
    
    with open(test_filename, "w+") as test_f:
        test_f.writelines(lines[train_idx:test_idx])

    with open(val_filename, "w+") as val_f:
        val_f.writelines(lines[test_idx:])
    

In [8]:
#KaggleDBQA https://github.com/Chia-Hsuan-Lee/KaggleDBQA

import os
import json

with open("cleaned_data/KaggleDBQA_combined.txt", "w+") as combined_file:
    for file_name_json in os.listdir("KaggleDBQA/examples"):
        if file_name_json[:-5].endswith("test") or file_name_json[:-5].endswith("fewshot"):
            continue
        with open(os.path.join("KaggleDBQA/examples", file_name_json), 'r') as file_json, \
             open("cleaned_data/" + file_name_json[:-5] + ".txt", "w+") as file_txt:
                data = json.load(file_json)
                for elem in data:
                    query: str = elem["query"]
                    file_txt.write(query + "\n")
                    combined_file.write(query + "\n")
                    

In [8]:
# for all non-clickhouse data we need to replace "" with '' because clickhouse does not support "" for string literals
import os

for walk_obj in os.walk("cleaned_data"):
    if len(walk_obj[1]) != 0:
        continue
    folder_name = walk_obj[0]
    for file_name in walk_obj[2]:
        file_name = os.path.join(folder_name, file_name)
        if file_name.lower().count("clickhouse") > 0:
            continue
    
        text = ""
        with open(file_name, "r") as f:
            text = f.read()
        
        with open(file_name, "w") as f:
            f.write(text.replace("\"", "\'"))
    



In [4]:
# synthetic_text_to_sql https://huggingface.co/datasets/gretelai/synthetic_text_to_sql/tree/main
import pandas as pd
from sklearn.model_selection import train_test_split

df_parquet = pd.read_parquet('synthetic_text_to_sql_train.snappy.parquet')
df_parquet_test = pd.read_parquet('synthetic_text_to_sql_test.snappy.parquet')


df_train, df_validation = train_test_split(df_parquet, test_size=0.07, random_state=42)


with open('cleaned_data/train/synth_train.txt', "w+") as file:
    for line in df_train['sql']:
        file.write(line + "\n")


with open('cleaned_data/test/synth_test.txt', "w+") as file:
    for line in df_parquet_test['sql']:
        file.write(line + "\n")


with open('cleaned_data/validation/synth_validation.txt', "w+") as file:
    for line in df_validation['sql']:
        file.write(line + "\n")